# **0. 대회 소개**


- 시계열 예측을 사용하여 에콰도르 기반의 대형 식료품 소매업체인 Corporación Favorita의 `매장 판매량`을 예측하고자 함
  - Favorita 매장에서 판매되는 수천 개 제품들의 `단위 판매량`을 제품군 별로 더 정확하게 예측하는 모델을 구축하고자 함


**평가 지표**
- Root Mean Squared Logarithmic Error(`RMSLE`)
  - 다음과 같이 계산  
  $ \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2} $
  - $n$: 총 인스턴스 수
  - $ \hat{y}_i$: 인스턴스 $i$에 대한 타겟의 예측값
  - $y_i$: 인스턴스 $i$에 대한 타겟의 실제값

# **1. 데이터 살펴보기**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format # 소수점 둘째자리까지만 표시

import warnings
warnings.filterwarnings('ignore')

## **1-1. 훈련 데이터**
- 날짜, 매장 및 제품 정보, 해당 제품이 프로모션되었는지 여부, 그리고 매출 숫자가 포함되어 있음
- 추가 파일에는 모델을 구축하는 데 유용할 수 있는 보충 정보가 포함되어 있음



### **a) train.csv**  
- 학습용 데이터  

**컬럼 설명**   
- `id`: 각 데이터를 구분하기 위한 식별자
  - 300만 건의 데이터
- `date`: 판매일자
  - 기간) 2013/01/01 ~ 2017/08/15
- `store_nbr`: 매장 고유 식별번호
  - 1 ~ 54번
- `family`: 판매되는 제품 유형
  - 33개의 제품군
- `sales`: 특정 날짜에 특정 매장의 특정 제품군의 총 매출
  - 제품은 분수 단위로 판매될 수 있기 때문에 소수 값이 가능  
    (예: 1.5 kg의 치즈, 1 봉지의 감자칩 등)  
  - **target 변수**
- `onpromotion`: 특정 날짜에 매장에서 프로모션되고 있는 제품군의 총 가짓수

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 로드
train=pd.read_csv("/content/drive/MyDrive/kaggle_storesales/train.csv")
oil=pd.read_csv("/content/drive/MyDrive/kaggle_storesales/oil.csv")
stores=pd.read_csv("/content/drive/MyDrive/kaggle_storesales/stores.csv")
test=pd.read_csv("/content/drive/MyDrive/kaggle_storesales/test.csv")

### **b) stores.csv**
- 매장 메타데이터  

**컬렴 설명**
  - `store_nbr`: 매장 고유 식별번호
  - `city`: 매장이 위치한 도시
    - 22개 도시
  - `state`: 매장이 위치한 주
    - 16개의 주
  - `type`: 매장 유형
    - A, B, C, D, E
  - `cluster`: 유사한 매장들의 그룹
    - 1 ~ 17

### **c) oil.csv**
- 일일 유가
  - 학습 및 테스트 데이터 기간 모두의 값을 포함
- 에콰도르는 석유 의존국임
  - 석유 가격의 변동에 매우 취약함
  - 이를 통해 어떤 제품군이 유가에 영향을 받는지 파악할 수 있음

**컬럼 설명**  
- `date`: 관측 일자
  - 기간) 2013/01/01 ~ 2017/08/31
- `dcoilwtico`: 유가

### **d) transaction.csv**
- 거래 데이터로, 하루 동안 매장에 방문한 사람 수 또는 하루에 생성된 송장(영수증) 수를 의미
  - 학습 데이터의 매출(sales) 컬럼과 매우 상관 관계가 있음
  - 이를 통해 매장의 매출 패턴을 이해할 수 있음
- `date`: 일자
  - 2013/01/01 ~ 2017/08/15
- `store_nbr`: 가게 고유번호
- `transactions`: 거래량

In [5]:
transaction = pd.read_csv("/content/drive/MyDrive/kaggle_storesales/transactions.csv")
transaction.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [6]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


### **e) holidays_events.csv**
- 휴일 및 이벤트 메타데이터
- `date`: 일자
  - 2012/03/02 ~ 2017/12/26
- `type` : 공휴일 속성
  - Holiday, Event, Additional, Transfer, Bridge, Work Day
- `locale`: 규모
  - National, Local, Regional
- `locale_name`: 지역명
  - 24개 지역
- `description`: 설명
- `transferred`: 대체 여부
  - True, False


In [12]:
holidays_events =pd.read_csv("/content/drive/MyDrive/kaggle_storesales/holidays_events.csv")
holidays_events.head(10)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False


In [8]:
# 2013-01-01 이전 데이터는 필요 없기에 삭제
holidays_events = holidays_events.loc[holidays_events['date'] > '2012-12-31'].reset_index(drop=True)

# (테스트 데이터까지 감안하여) 2017-08-31 이후 데이터는 필요 없기에 삭제
holidays_events = holidays_events.loc[holidays_events['date'] < '2017-08-31'].reset_index(drop=True)

In [13]:
### 변수명 변경
# stores.csv의 type과 변수명이 중복됨

holidays_events = holidays_events.rename(columns={'type': 'category'})

In [14]:
holidays_events['category'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [15]:
holidays_events['locale'].unique()

array(['Local', 'Regional', 'National'], dtype=object)

#### **📌 caterogy 처리**

**Transfer**

In [16]:
# transferred == True인 행을 drop
holidays_events = holidays_events.loc[holidays_events['transferred'] != True].reset_index(drop=True) # False만 택함

**Bridge & Work Day**

In [ ]:
# category == Work Day 인 행을 drop 하지 않고 일단 살려두기
# holidays_events = holidays_events.loc[holidays_events['category'] != 'Work Day'].reset_index(drop=True)

#### **📌 locale 처리**
- 중복된 날짜가 있는 경우 병합 시 문제가 생길 가능성이 높음
  - 중복된 데이터를 확인 후 최대한 중복 행 제거

In [17]:
## 국가 공휴일
# 지역명 == 'Ecuador'
print(holidays_events.loc[holidays_events['locale'] == 'National', 'locale_name'].unique())
print(len(holidays_events.loc[holidays_events['locale'] == 'National', 'locale_name'].unique()))

['Ecuador']
1


In [18]:
## 지역(주) 공휴일
# 지역명 == 주(state) 명
print(holidays_events.loc[holidays_events['locale'] == 'Regional', 'locale_name'].unique())
print(len(holidays_events.loc[holidays_events['locale'] == 'Regional', 'locale_name'].unique()))

['Cotopaxi' 'Imbabura' 'Santo Domingo de los Tsachilas' 'Santa Elena']
4


In [19]:
## 지역(도시) 공휴일
# 지역명 == 도시(city) 명
print(holidays_events.loc[holidays_events['locale'] == 'Local', 'locale_name'].unique())
print(len(holidays_events.loc[holidays_events['locale'] == 'Local', 'locale_name'].unique()))

['Manta' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda' 'Latacunga'
 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe' 'Esmeraldas' 'Ambato'
 'Ibarra' 'Quevedo' 'Quito' 'Loja' 'Salinas' 'Guayaquil']
19


In [20]:
holidays_events

,date,category,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
333,2017-12-22,Additional,National,Ecuador,Navidad-3,False
334,2017-12-23,Additional,National,Ecuador,Navidad-2,False
335,2017-12-24,Additional,National,Ecuador,Navidad-1,False
336,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [21]:
holidays_events.drop(['description','transferred'],axis=1,inplace=True)

In [22]:
holidays_events.head()

,date,category,locale,locale_name
0,2012-03-02,Holiday,Local,Manta
1,2012-04-01,Holiday,Regional,Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca
3,2012-04-14,Holiday,Local,Libertad
4,2012-04-21,Holiday,Local,Riobamba


In [23]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


In [24]:
train.shape

(3000888, 6)

In [25]:
holidays_events['date']

0      2012-03-02
1      2012-04-01
2      2012-04-12
3      2012-04-14
4      2012-04-21
          ...    
333    2017-12-22
334    2017-12-23
335    2017-12-24
336    2017-12-25
337    2017-12-26
Name: date, Length: 338, dtype: object

In [26]:
## 중복된 날짜 확인
holidays_events[holidays_events.duplicated(subset = 'date', keep = False)]

,date,category,locale,locale_name
7,2012-06-25,Holiday,Regional,Imbabura
8,2012-06-25,Holiday,Local,Latacunga
9,2012-06-25,Holiday,Local,Machala
10,2012-07-03,Holiday,Local,Santo Domingo
11,2012-07-03,Holiday,Local,El Carmen
...,...,...,...,...
310,2017-07-03,Holiday,Local,Santo Domingo
329,2017-12-08,Holiday,Local,Loja
330,2017-12-08,Transfer,Local,Quito
332,2017-12-22,Holiday,Local,Salinas


In [27]:
holidays_events[holidays_events.duplicated(subset = 'date', keep = False)]['date'].unique()

array(['2012-06-25', '2012-07-03', '2012-12-22', '2012-12-24',
       '2012-12-31', '2013-05-12', '2013-06-25', '2013-07-03',
       '2013-12-22', '2014-06-25', '2014-07-03', '2014-12-22',
       '2014-12-26', '2015-06-25', '2015-07-03', '2015-12-22',
       '2016-04-21', '2016-05-01', '2016-05-07', '2016-05-08',
       '2016-05-12', '2016-06-25', '2016-07-03', '2016-07-24',
       '2016-11-12', '2016-12-22', '2017-04-14', '2017-06-25',
       '2017-07-03', '2017-12-08', '2017-12-22'], dtype=object)

## **1-2. 테스트 데이터**
- 기간) 2017/08/16 ~ 2017/08/31

In [28]:
test = pd.read_csv("/content/drive/MyDrive/kaggle_storesales/test.csv")
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [29]:
test.tail()

,id,date,store_nbr,family,onpromotion
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


## **1-3. 제출용 파일**

In [31]:
sample_submission = pd.read_csv("/content/drive/MyDrive/kaggle_storesales/sample_submission.csv")
sample_submission.head()

,id,sales
0,3000888,0.00
1,3000889,0.00
2,3000890,0.00
3,3000891,0.00
4,3000892,0.00


# **2. 데이터 병합**
- `train`의 모든 행은 결과 데이터프레임에 포함되며, 병합하려는 데이터프레임의 정보는 병합 대상인 열을 기준으로 매칭되는 경우에만 병합

## **2-1. train + stores**
- `store_nbr`을 기준으로 병합

In [32]:
# 매장 고유번호를 기준으로 merge
train = pd.merge(train, stores, on = 'store_nbr', how = 'left')
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13


In [33]:
train.shape

(3000888, 10)

In [34]:
## test
test = pd.merge(test, stores, on = 'store_nbr', how = 'left')
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13


## **2-2. train + oil**
- `date` 열을 기준으로 병햡

In [37]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [36]:
oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [38]:
# oil 데이터에 na값을 뒷값으로 채우기 (fillna 사용)
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

In [45]:
oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [40]:
# 날짜를 기준으로 merge
train = pd.merge(train, oil, on = 'date', how = 'left')
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14


In [41]:
train.shape

(3000888, 11)

In [43]:
train['dcoilwtico'].isna().sum()

857142

In [50]:
# train 데이터 dcoilwtico에서 누락된 값을 위아래 선형평균으로 대체하기
train['dcoilwtico']=train['dcoilwtico'].interpolate()

In [51]:
train['dcoilwtico'].isna().sum()
# 누락된 값이 없음을 확인

0

In [52]:
## test
test = pd.merge(test, oil, on = 'date', how = 'left')
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.80
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.80
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.80
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.80
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.80


In [53]:
test['dcoilwtico'].isna().sum()

7128

In [55]:
# test 데이터 dcoilwtico에서 누락된 값을 위아래 선형평균으로 대체하기
test['dcoilwtico']=test['dcoilwtico'].interpolate()
# test 데이터 dcoilwtico 에서 누락된 값이 없음을 확인
test['dcoilwtico'].isna().sum()

0

## **2-3. train + transaction**
- EDA 시에는 활용 가능
- 모델 학습 시에는 활용하기 어려울 듯
  - test 데이터가 수집된 시점에 대해 거래 데이터가 없음
  - 모델 학습 시에는 해당 변수를 drop 해야 함
- transaction의 PK는 (date, store_nbr)임
  - 두 개를 쌍으로 고려해야 각각의 데이터(행) 구분 가능
  - `date`, `store_nbr`을 기준으로 결합

In [56]:
# 날짜와 가게 고유 번호를 기준으로 merge
train = pd.merge(train, transaction, on = ['date', 'store_nbr'], how = 'left')
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,NaN
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14,NaN
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14,NaN
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14,NaN
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14,NaN


In [57]:
train['transactions'].isna().sum()

245784

In [58]:
train.shape

(3000888, 12)

- transaction 데이터의 경우 test에는 합칠 수 없음

## **2-4. train + holidays_events**



### **a) train데이터에 요일 변수 추가**

In [ ]:
train.head()

In [66]:
from datetime import datetime

# 'YY-MM-DD' 형식의 문자열을 날짜로 변환
train['date2'] = pd.to_datetime(train['date'], format='%Y-%m-%d')

# 날짜 열에서 요일 열을 파생
train['weekday'] = train['date2'].dt.day_name()

# date2 칼럼 삭세
train.drop('date2',inplace=True,axis=1)

In [67]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,weekday
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday


In [68]:
# test 데이터에도 똑같이 전처리

# 'YY-MM-DD' 형식의 문자열을 날짜로 변환
test['date2'] = pd.to_datetime(test['date'], format='%Y-%m-%d')

# 날짜 열에서 요일 열을 파생
test['weekday'] = test['date2'].dt.day_name()

# date2 칼럼 삭세
test.drop('date2',inplace=True,axis=1)

train['weekday']에서 주말이면 1, 평일이면 0으로 하는 'weekend'변수 파생
- 이때 holidays_events에서 category가 bridge이면 weekend 변수 1로 변환
- 또한 holidays_events에서 category가 workdays이면 weekend 변수 0으로 변환

In [69]:
train['weekday'].value_counts()

Tuesday      431244
Saturday     429462
Monday       429462
Wednesday    427680
Thursday     427680
Friday       427680
Sunday       427680
Name: weekday, dtype: int64

In [77]:
# train 데이터에서 weekend 변수 만들기
train['weekend']=0
train.loc[train['weekday'].isin(['Sunday','Saturday']),'weekend']=1

# test 데이터에서 weekend 변수 만들기
test['weekend']=0
test.loc[test['weekday'].isin(['Sunday','Saturday']),'weekend']=1

In [80]:
holidays_events[holidays_events['category']=='Bridge']

,date,category,locale,locale_name
34,2012-12-24,Bridge,National,Ecuador
38,2012-12-31,Bridge,National,Ecuador
153,2014-12-26,Bridge,National,Ecuador
157,2015-01-02,Bridge,National,Ecuador
271,2016-11-04,Bridge,National,Ecuador


In [83]:
holidays_events[holidays_events['category']=='Work Day']

,date,category,locale,locale_name
41,2013-01-05,Work Day,National,Ecuador
42,2013-01-12,Work Day,National,Ecuador
146,2014-12-20,Work Day,National,Ecuador
158,2015-01-10,Work Day,National,Ecuador
277,2016-11-12,Work Day,National,Ecuador


In [86]:
# 다행히 category가 Bridge인 열과 Workday인 열 모두 train 데이터에 있음
train.loc[train['date'].isin(holidays_events[holidays_events['category']=='Bridge']['date']),'weekend']=1
train.loc[train['date'].isin(holidays_events[holidays_events['category']=='Work Day']['date']),'weekend']=0

In [88]:
train['weekend'].value_counts()

0    2147310
1     853578
Name: weekend, dtype: int64

holidays_events에서 category 변수 삭제

In [90]:
holidays_events.drop('category',inplace=True,axis=1)

### **b) locale에 따라 for문 사용하여 merge**

📌 **국가 공휴일인 경우(locale = 'National')**  
- `date` 기준으로 병합

📌 **지역(주) 공휴일인 경우(locale = 'Regional')**
- `date`와 `state` 기준으로 병합

📌 **지역(도시) 공휴일인 경우(locale = 'Local')**
- `date`와 `city` 기준으로 병합

In [91]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,weekday,weekend
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0


In [92]:
holidays_events

,date,locale,locale_name
0,2012-03-02,Local,Manta
1,2012-04-01,Regional,Cotopaxi
2,2012-04-12,Local,Cuenca
3,2012-04-14,Local,Libertad
4,2012-04-21,Local,Riobamba
...,...,...,...
333,2017-12-22,National,Ecuador
334,2017-12-23,National,Ecuador
335,2017-12-24,National,Ecuador
336,2017-12-25,National,Ecuador


In [98]:
# train holiday' 변수 초기화 (기본값: 0)
train['holiday'] = 0
# holidays events의 locale에 따라 'holiday' 값을 수정
for index,row in holidays_events.iterrows():
  date_condition=train['date']==row['date']
  if row['locale'] == 'National':
    train.loc[date_condition,'holiday'] = 1
  elif row['locale'] == 'Regional':
    locale_condition = train['state'] == row['locale_name']
    train.loc[date_condition & locale_condition,'holiday'] = 1
  elif row['locale'] == 'Local':
    locale_condition = train['city'] == row['locale_name']
    train.loc[date_condition & locale_condition,'holiday'] = 1

In [99]:
# 결과 확인
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,weekday,weekend,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1


In [105]:
train['holiday'].value_counts()

0    2746128
1     254760
Name: holiday, dtype: int64

In [102]:
# test 데이터에도 똑같이 전처리
# test 'holiday' 변수 초기화 (기본값: 0)
test['holiday'] = 0
# holidays events의 locale에 따라 'holiday' 값을 수정
for index,row in holidays_events.iterrows():
  date_condition=test['date']==row['date']
  if row['locale'] == 'National':
    test.loc[date_condition,'holiday'] = 1
  elif row['locale'] == 'Regional':
    locale_condition = test['state'] == row['locale_name']
    test.loc[date_condition & locale_condition,'holiday'] = 1
  elif row['locale'] == 'Local':
    locale_condition = test['city'] == row['locale_name']
    test.loc[date_condition & locale_condition,'holiday'] = 1

In [103]:
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,weekday,weekend,holiday
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.80,Wednesday,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.80,Wednesday,0,0
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0


In [104]:
test['holiday'].value_counts()

0    28446
1       66
Name: holiday, dtype: int64

## **2-5. 병합 결과 확인**

In [106]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,weekday,weekend,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
1,1,2013-01-01,1,BABY CARE,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
2,2,2013-01-01,1,BEAUTY,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
3,3,2013-01-01,1,BEVERAGES,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1
4,4,2013-01-01,1,BOOKS,0.00,0,Quito,Pichincha,D,13,93.14,NaN,Tuesday,0,1


In [107]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 15 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   city          object 
 7   state         object 
 8   type          object 
 9   cluster       int64  
 10  dcoilwtico    float64
 11  transactions  float64
 12  weekday       object 
 13  weekend       int64  
 14  holiday       int64  
dtypes: float64(3), int64(6), object(6)
memory usage: 366.3+ MB


In [108]:
train.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type                 0
cluster              0
dcoilwtico           0
transactions    245784
weekday              0
weekend              0
holiday              0
dtype: int64

In [109]:
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,weekday,weekend,holiday
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.80,Wednesday,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.80,Wednesday,0,0
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.80,Wednesday,0,0


In [110]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           28512 non-null  int64  
 1   date         28512 non-null  object 
 2   store_nbr    28512 non-null  int64  
 3   family       28512 non-null  object 
 4   onpromotion  28512 non-null  int64  
 5   city         28512 non-null  object 
 6   state        28512 non-null  object 
 7   type         28512 non-null  object 
 8   cluster      28512 non-null  int64  
 9   dcoilwtico   28512 non-null  float64
 10  weekday      28512 non-null  object 
 11  weekend      28512 non-null  int64  
 12  holiday      28512 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 3.0+ MB


In [111]:
test.isna().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
city           0
state          0
type           0
cluster        0
dcoilwtico     0
weekday        0
weekend        0
holiday        0
dtype: int64

## **2-6. 병합된 파일 저장**

In [112]:
train.to_csv("/content/drive/MyDrive/kaggle_storesales/preproc_train.csv", index = False)
test.to_csv("/content/drive/MyDrive/kaggle_storesales/preproc_test.csv", index = False)